In [36]:
pip install sentence-transformers

  Using cached tokenizers-0.20.3-cp312-none-win_amd64.whl.metadata (6.9 kB)
   ---------------------------------------- 0.0/203.0 MB ? eta -:--:--
    --------------------------------------- 2.6/203.0 MB 12.5 MB/s eta 0:00:16
   - -------------------------------------- 5.2/203.0 MB 12.7 MB/s eta 0:00:16
   - -------------------------------------- 7.9/203.0 MB 12.8 MB/s eta 0:00:16
   -- ------------------------------------- 10.7/203.0 MB 12.7 MB/s eta 0:00:16
   -- ------------------------------------- 13.4/203.0 MB 12.7 MB/s eta 0:00:15
   --- ------------------------------------ 16.3/203.0 MB 12.8 MB/s eta 0:00:15
   --- ------------------------------------ 18.9/203.0 MB 12.8 MB/s eta 0:00:15
   ---- ----------------------------------- 21.5/203.0 MB 12.8 MB/s eta 0:00:15
   ---- ----------------------------------- 24.1/203.0 MB 12.8 MB/s eta 0:00:14
   ----- ---------------------------------- 27.0/203.0 MB 12.9 MB/s eta 0:00:14
   ----- ---------------------------------- 29.6/203.0 M

In [9]:
pip install llama-index-embeddings-huggingface


Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install llama-index-llms-openai

  Using cached llama_index_core-0.11.23-py3-none-any.whl.metadata (2.5 kB)
Using cached llama_index_core-0.11.23-py3-none-any.whl (1.6 MB)
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.12.2
    Uninstalling llama-index-core-0.12.2:
      Successfully uninstalled llama-index-core-0.12.2
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-embeddings-huggingface 0.4.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.11.23 which is incompatible.
llama-index-vector-stores-faiss 0.3.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.11.23 which is incompatible.


In [1]:
import os

# Setze deinen OpenAI API-Schlüssel
os.environ["OPENAI_API_KEY"] = "xy"

In [35]:
#pip install llama-index-vector-stores-faiss faiss-cpu
import os.path
import faiss
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    Settings,
    load_index_from_storage,
)
from llama_index.vector_stores.faiss import FaissVectorStore
#from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openai import OpenAI
from time import time

# check if storage already exists
PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader("data").load_data()
    Settings.chunk_size = 512
    Settings.chunk_overlap = 50
    
    Settings.llm = OpenAI(model="gpt-3.5-turbo")
    Settings.embed_model = HuggingFaceEmbedding(
        model_name="BAAI/bge-small-en-v1.5", top_n=3
    )

    embedding = Settings.embed_model

    # Verwende das SentenceTransformer-Modell für Embeddings
    embedding = SentenceTransformerEmbedding(model)

    # Erstelle den Index mit dem gewählten Embedding-Modell
    #index = VectorStoreIndex.from_documents(documents, embedding=embedding)

    # Erstellen des FAISS-basierten VectorStore
    d=1536
    faiss_index = faiss.IndexFlatL2(d)  # 512 ist die Dimension des Embeddings, 1536 für gpt-3.5-embeddings
    vector_store = FaissVectorStore(faiss_index=faiss_index)
    # Erstellen des Index mit FAISS als Speichersystem
    index = VectorStoreIndex.from_documents(documents=documents, vector_store=vector_store)
    #index = VectorStoreIndex.from_documents(documents, )

    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

# Either way we can now query the index
#query_engine = index.as_query_engine()
#query_engine = index.as_query_engine(similarity_top_k=4)
# Definiere die Abfrageengine
query_engine = index.as_query_engine(similarity_top_k=4)

# Frage ausführen
now = time()
response = query_engine.query("What is multicast?")
print(f"Elapsed: {round(time() - now, 2)}s")
print(response)
#response = query_engine.query("Can you please sum up the introduction to multicast?")
#print(response)

Elapsed: 1.71s
Multicast is a form of communication that involves coordination and agreement among group members to receive copies of messages sent to the group. It allows for different delivery guarantees, such as agreeing on the set of messages received or on delivery ordering. Multicast can be achieved through a single operation instead of sending messages individually to each member, which enhances efficiency by sending once on each link and utilizing hardware multicast when available.


In [5]:
from time import time

In [9]:
from llama_index.core.postprocessor import SentenceTransformerRerank

rerank = SentenceTransformerRerank(
    model="cross-encoder/ms-marco-MiniLM-L-2-v2", top_n=3
)

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

C:\Users\katha\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\katha\.cache\huggingface\hub\models--cross-encoder--ms-marco-MiniLM-L-2-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/62.5M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [13]:
query_engine = index.as_query_engine(
    similarity_top_k=10, node_postprocessors=[rerank]
)

now = time()
response = query_engine.query(
    "What is multicast?",
)
print(f"Elapsed: {round(time() - now, 2)}s")

Elapsed: 1.96s


In [13]:
query_engine = index.as_query_engine(
    similarity_top_k=10, node_postprocessors=[rerank]
)

now = time()
response = query_engine.query(
    "What is multicast?",
)
print(f"Elapsed: {round(time() - now, 2)}s")

Elapsed: 1.96s


In [19]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [43]:
#pip install llama-index-vector-stores-faiss faiss-cpu
import os.path
import faiss
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)
from llama_index.vector_stores.faiss import FaissVectorStore

# check if storage already exists
PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader("data").load_data()
    Settings.chunk_size = 512
    Settings.chunk_overlap = 50
    
    # Erstellen des FAISS-basierten VectorStore
    d=1536
    faiss_index = faiss.IndexFlatL2(d)  # 512 ist die Dimension des Embeddings, 1536 für gpt-3.5-embeddings
    vector_store = FaissVectorStore(faiss_index=faiss_index)
    # Erstellen des Index mit FAISS als Speichersystem
    index = VectorStoreIndex.from_documents(documents, vector_store=vector_store)
    #index = VectorStoreIndex.from_documents(documents, )

    #index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

# Either way we can now query the index
#query_engine = index.as_query_engine()

query_engine = index.as_query_engine(similarity_top_k=4)

# Frage ausführen
now = time()
response = query_engine.query("What is multicast?")
print(f"Elapsed: {round(time() - now, 2)}s")
print(response)

#query_engine = index.as_query_engine(similarity_top_k=4)
#response = query_engine.query("Can you please sum up the introduction to multicast?")
#print(response)

Elapsed: 1.58s
Multicast is a form of communication that involves coordination and agreement among members of a group to receive copies of messages sent to the group. It allows for different delivery guarantees, such as agreeing on the set of messages received or on delivery ordering. Multicast can be achieved through a single operation instead of sending messages individually to each member, which enhances efficiency by sending once on each link and utilizing hardware multicast when available.


In [17]:
response = query_engine.query("Why do I have to use multicast?")

In [18]:
print(response)

Using multicast is beneficial because it allows for efficient communication by sending messages once on each link, utilizing hardware multicast when available. This method is particularly advantageous when transmitting data from one source to multiple destinations, reducing network traffic and improving overall performance.


In [39]:
response = query_engine.query("Can you please complete the following sentence: Multicast communication requires...")

In [41]:
print(response)

Coordination and agreement.


In [45]:
response = query_engine.query("Kannst du mir die Folie ausgeben, auf welcher dieses Ergebnis steht?")

In [47]:
print(response)

Page 11.


In [51]:
response = query_engine.query("Nutzt du zum Abfragen und Antworten nur das angebundene PDF?")
print(response)

Ja, ich nutze nur das angebundene PDF zum Abfragen und Antworten.


In [55]:
response = query_engine.query("pourquoi utilisez-vous les PDF liés ?")
print(response)

PDFs are used for their ability to store and display information in a consistent format across different devices and platforms.


In [57]:
response = query_engine.query("Sicher, dass du Seite 11 meintest?")
print(response)

Yes, I am sure that Seite 11 was meant.


In [ ]:
response = query_engine.query("Can you please tell me what information I can find on page 11")
print(response)

You can find information about the revision of IP multicast on page 11.


In [61]:
response = query_engine.query("What did I asked you?")
print(response)

You asked about the content provided in the context information.


In [75]:
pip install llama-index-vector-stores-faiss faiss-cpu

Note: you may need to restart the kernel to use updated packages.
